# Get things ready

In [ ]:
import importlib
import numpy as np
import numpy.typing as npt
from data_exploration.helpers import find_file, save
from tqdm import tqdm
import pandas as pd
import matplotlib.pyplot as plt

import os
os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'
import trackml_copy as outrunner_code
import trackml_2_solution_example as my_code

DO_EXPORT = True
DIRECTORY = my_code.DIRECTORY
SOLUTION_DIR = my_code.SOLUTION_DIR


DATA_ROOT = "/data/atlas/users/lschoonh/BachelorProject/data/"
DATA_SAMPLE = DATA_ROOT + "train_100_events/"
MODELS_ROOT = DIRECTORY + "trained_models/2nd_place/"


Start from same point

In [ ]:
event_name: str = "event000001001"
hits, cells, truth, particles = outrunner_code.get_event(event_name)
preload = True

my_preds: list[npt.NDArray] = find_file(f"preds_{event_name}", dir=DIRECTORY)  # type: ignore
outrunner_preds = np.load(SOLUTION_DIR + "my_%s.npy" % event_name, allow_pickle=True)
module_id = my_code.get_module_id(hits)
PATH_THR = 0.85

# Debug

## Compare paths

Just run get_path

In [ ]:
hit_index = 0

mask = np.ones(len(hits))

# My path uses id, not index
my_path_0 = my_code.get_path(hit_index + 1, thr=PATH_THR, mask=mask, module_id=module_id,preds=outrunner_preds) -1 
outrunner_path_0 = outrunner_code.get_path2(hit_index, thr=PATH_THR, mask=mask, module_id=module_id, preds=outrunner_preds)

print("my path", my_path_0[:10])
print("outrunner path", outrunner_path_0[:10])
print("Instances not in agreement: ", np.where(my_path_0 != outrunner_path_0)[0])


Run with my preds

In [ ]:
hit_index = 0

mask = np.ones(len(hits))

# My path uses id, not index
my_path_0 = my_code.get_path(hit_index + 1, thr=PATH_THR, mask=mask, module_id=module_id,preds=my_preds) -1 
outrunner_path_0 = outrunner_code.get_path2(hit_index, thr=PATH_THR, mask=mask, module_id=module_id, preds=outrunner_preds)

print("my path", my_path_0[:10])
print("outrunner path", outrunner_path_0[:10])
print("Instances not in agreement: ", np.where(my_path_0 != outrunner_path_0)[0])


Get n paths

In [ ]:
debug_limit = 1000 # generate only n paths
my_new_tracks_limited = my_code.get_all_paths(hits, thr= PATH_THR, preds=outrunner_preds, module_id=module_id, debug_limit=debug_limit)

outrunner_tracks_regenerated_limited: list[npt.NDArray] = outrunner_code.get_all_paths(hits, thr=PATH_THR, preds=outrunner_preds, module_id=module_id, debug_limit=debug_limit)

Print first n' tracks

In [ ]:
limit = debug_limit
print(f"{limit} my tracks", my_new_tracks_limited[:limit])
print(f"{limit} outrunner tracks", outrunner_tracks_regenerated_limited[:limit])

Compare tracks

In [ ]:
for test_track, verification_tracks in zip(my_new_tracks_limited[debug_limit], outrunner_tracks_regenerated_limited[debug_limit]):
    assert np.all(test_track-1 == verification_tracks), "Tracks are not equal"
print(f"All first {debug_limit} tracks are equal")

Compare first track

In [ ]:
# Subtrackt 1 from all ids to get indices
my_track_0 = my_new_tracks_limited[0]-1
outrunner_track_0 = outrunner_tracks_regenerated_limited[0]

print("my 1st tracks", my_track_0)
print("outrunner 1st tracks", outrunner_track_0)

Show outliers

In [ ]:
# print('isin' ,np.isin(my_track_0, outrunner_track_0, ))
overlap = np.where(np.isin(my_track_0, outrunner_track_0) == True)
outliers_1= np.where(np.isin(my_track_0, outrunner_track_0) == False)
outliers_2= np.where(np.isin(outrunner_track_0, my_track_0) == False)
print("overlap", overlap)
print("my hits not in outrunner", outliers_1)
print("outrunner hits not in mine", outliers_2)


## Compare preds

In [ ]:
outrunner_model = my_code.load_model(SOLUTION_DIR + "my_model.h5")

In [ ]:
hits, cells, truth, particles = outrunner_code.get_event(event_name)
hit_cells = cells.groupby(['hit_id']).value.count().values
hit_value = cells.groupby(['hit_id']).value.sum().values
hit_value = cells.groupby(['hit_id']).value.sum().values
outrunner_features = np.hstack((hits[['x','y','z']]/1000, hit_cells.reshape(len(hit_cells),1)/10,hit_value.reshape(len(hit_cells),1)))

Run just get_predict for both

In [ ]:
hit_index = 0
my_p_0 = my_code.retrieve_predict(hit_index+1, preds=outrunner_preds)
out_p_0 = outrunner_code.retrieve_predict(hit_index, preds=outrunner_preds)
print("my p", my_p_0[:15])
print("out p", out_p_0[:15])
print("Instances not in agreement: ", np.where(my_p_0 != out_p_0)[0])

In [ ]:
hit_id = 1

In [ ]:
idx_0 = outrunner_preds[0][0]
idx_0

In [ ]:
outrunner_preds[0][1]


In [ ]:
outrunner_preds[0][1][0]

In [ ]:
p_my = my_code.make_predict(model=outrunner_model,features=outrunner_features, hits=hits, hit_id=hit_id, thr=0.85)
p_my[8]

Try getting preds for hit_index = 0

In [ ]:
batch_size = 200
i = hit_id -1
model = outrunner_model

# ------------------ 1st part ------------------

TestX = np.zeros((len(outrunner_features), 10))
TestX[:,5:] = outrunner_features

# for TTA
TestX1 = np.zeros((len(outrunner_features), 10))
TestX1[:,:5] = outrunner_features

preds = []

TestX[i+1:,:5] = np.tile(outrunner_features[i], (len(TestX)-i-1, 1))
pred = model.predict(TestX[i+1:], batch_size=batch_size, verbose="0")[:,0]                
idx = np.where(pred>0.2)[0]


if len(idx) > 0:
    TestX1[idx+i+1,5:] = TestX[idx+i+1,:5]
    pred1 = model.predict(TestX1[idx+i+1], batch_size=batch_size, verbose="0")[:,0]
    pred[idx] = (pred[idx]+pred1)/2

idx = np.where(pred>0.5)[0]

p = np.zeros(len(pred), dtype=np.int32)
# [idx+i+1, pred[idx]]
# p[idx+i+1] = pred[idx]
indices = idx+i+1
p[indices] = pred[idx]

In [ ]:
pred[idx]

In [ ]:
pred[7]

In [ ]:
p_outrunner = my_code.retrieve_predict(hit_id, outrunner_preds)
p_outrunner

In [ ]:
p_outrunner_re = outrunner_code.get_predict(hit_index=hit_id-1, hits=hits,model=outrunner_model, features=outrunner_features)

Get preds by preds_matrix

In [ ]:
thr=0.85
my_code.get_path(hit_id, thr=thr,mask=np.ones(len(hits)), module_id=module_id, preds=outrunner_preds)

Get preds by predicting

In [ ]:
my_code.get_path(hit_id, thr=thr,mask=np.ones(len(hits)), module_id=module_id, features=outrunner_features, model=outrunner_model, hits=hits)

## Compare scores

In [ ]:

used_tracks = [outrunner_tracks, outrunner_tracks_regenerated][1]
limit = None

_make_my_scores = lambda: save(
    my_get_track_scores(used_tracks, 8, limit, index_shift=0),
    name="my_scores",
    tag=event_name,
    prefix=DIRECTORY,
    save=DO_EXPORT,
)
_make_outrunner_scores = lambda: save(
    outrunner_get_track_scores(used_tracks, 8, limit),
    name="outrunner_scores",
    tag=event_name,
    prefix=DIRECTORY,
    save=DO_EXPORT,
)

my_scores: npt.NDArray = find_file("my_scores", dir=DIRECTORY, fallback_func=_make_my_scores, force_fallback=not preload)  # type: ignore

outrunner_scores: npt.NDArray = find_file("outrunner_scores", dir=DIRECTORY, fallback_func=_make_outrunner_scores, force_fallback=not preload)  # type: ignore

limit = 5 if limit is None else limit
print("My scores: ", my_scores[:limit])
print("Outrunner scores: ", outrunner_scores[:limit])

pass

# Debug grand scale

In [ ]:
def verify_matrices(test_matrix: npt.NDArray | list[npt.NDArray], verification_matrix: npt.NDArray | list[npt.NDArray], limit: int | None = None):
    for i, (test_row, verification_row) in tqdm(enumerate(zip(test_matrix, verification_matrix)), total=min(len(test_matrix), len(verification_matrix))):
        if limit is not None and i >= limit:
            break

        tracks_equal = np.all(test_row == np.array(verification_row))
        if not tracks_equal:
            print("Rows are not equal")
            print("test row", test_row)
            print("good row", verification_row)
            print("Instances not in agreement: ", np.where(test_row!= verification_row)[0])
            raise ValueError("Rows are not equal")
    print(f"All first {i+1} rows are equal")
    return True

## Compare predictions

Start from same point

In [ ]:
hits, cells, truth, particles = outrunner_code.get_event(event_name)
hit_cells = cells.groupby(['hit_id']).value.count().values
hit_value = cells.groupby(['hit_id']).value.sum().values
hit_value = cells.groupby(['hit_id']).value.sum().values


### Make features

In [ ]:
outrunner_features = np.hstack((hits[['x','y','z']]/1000, hit_cells.reshape(len(hit_cells),1)/10,hit_value.reshape(len(hit_cells),1)))
outrunner_features

In [ ]:
my_features = my_code.get_featured_event(event_name).features
my_features

In [ ]:
verify_matrices(my_features, outrunner_features, limit=1000)

### Make preds

In [ ]:
my_preds[0]

In [ ]:
pd.DataFrame(my_preds[0])

In [ ]:
pd.DataFrame(outrunner_preds[0][0],outrunner_preds[0][1])
# verify_matrices(my_preds, outrunner_preds)

Load model

In [ ]:
outrunner_model = my_code.load_model(SOLUTION_DIR + "my_model.h5")

In [ ]:
used_features = outrunner_features
used_model = outrunner_model
pred_matrix_limit = 100

In [ ]:
my_pred_matrix = my_code.make_predict_matrix(model=used_model,features=used_features, debug_limit=pred_matrix_limit)

In [ ]:
my_pred_matrix

In [ ]:
TestX = np.zeros((len(used_features), 10))
TestX[:,5:] = used_features

# for TTA
TestX1 = np.zeros((len(used_features), 10))
TestX1[:,:5] = used_features

preds = []

for i in tqdm(range(pred_matrix_limit)):
    TestX[i+1:,:5] = np.tile(used_features[i], (len(TestX)-i-1, 1))

    pred = used_model.predict(TestX[i+1:], batch_size=20000,verbose="0")[:,0]                
    idx = np.where(pred>0.2)[0]

    if len(idx) > 0:
        TestX1[idx+i+1,5:] = TestX[idx+i+1,:5]
        pred1 = used_model.predict(TestX1[idx+i+1], batch_size=20000,verbose="0")[:,0]
        pred[idx] = (pred[idx]+pred1)/2

    idx = np.where(pred>0.5)[0]

    preds.append([idx+i+1, pred[idx]])

    #if i==0: print(preds[-1])

preds.append([np.array([], dtype='int64'), np.array([], dtype='float32')])

In [ ]:
verify_matrices(outrunner_preds[0], preds[0], limit=100)

In [ ]:

# rebuild to NxN
for i in range(len(preds)):
    ii = len(preds)-i-1
    for j in range(len(preds[ii][0])):
        jj = preds[ii][0][j]
        preds[jj][0] = np.insert(preds[jj][0], 0 ,ii)
        preds[jj][1] = np.insert(preds[jj][1], 0 ,preds[ii][1][j])

In [ ]:
outrunner_pred_matrix = outrunner_code.make_predict_matrix(model=used_model,features=used_features, debug_limit=pred_matrix_limit)

## Compare tracks

Define some functions that produce and then save result

In [ ]:
# Predict paths
_make_tracks = lambda: save(
    my_code.get_all_paths(hits, PATH_THR, module_id=module_id, preds=outrunner_preds, do_redraw=True),
    name="new_tracks_all",
    tag=event_name,
    prefix=DIRECTORY,
    save=DO_EXPORT
)


# calculate track's confidence
_make_scores = lambda: save(
    my_code.get_track_scores(tracks_all), name="new_scores", tag=event_name, prefix=DIRECTORY, save=DO_EXPORT
)

# Merge seeds to definite id's
_make_merged_tracks = lambda: save(
    my_code.run_merging(tracks_all, scores, preds=outrunner_preds, multi_stage=True, module_id=module_id,log_evaluations=True, truth=truth),
    name="merged_tracks",
    tag=event_name,
    prefix=DIRECTORY,
    save=DO_EXPORT,
)  # type: ignore

# Save submission
_make_submission = lambda: save(
    pd.DataFrame({"hit_id": hits.hit_id, "track_id": merged_tracks}),
    name="submission",
    tag=event_name,
    prefix=DIRECTORY,
    save=do_export,
)

In [ ]:
# Keep in mind result is not loaded if `preload` is False
print(f"Preload: {preload}")

Get tracks

In [ ]:
tracks_all: list[npt.NDArray] = find_file(f"new_tracks_all_{event_name}", dir=DIRECTORY, fallback_func=_make_tracks, force_fallback=not preload)  # type: ignore
outrunner_tracks: list[npt.NDArray] = find_file(f"outrunner_tracks_all", dir=DIRECTORY, extension='pkl')  # type: ignore

Verify

In [ ]:
test_tracks = tracks_all
verification_tracks = outrunner_tracks # outrunner_tracks_regenerated_limited

for i, (test_track, verification_tracks) in tqdm(enumerate(zip(test_tracks, verification_tracks)), total=min(len(test_tracks), len(verification_tracks))):
    verification_tracks = np.array(verification_tracks)
    test_track = np.array(test_track) - 1
    tracks_equal = np.all(test_track == np.array(verification_tracks))
    if not tracks_equal:
        print("Tracks are not equal")
        print("test track", test_track)
        print("good track", verification_tracks)
        print("Instances not in agreement: ", np.where(test_track!= verification_tracks)[0])
        raise ValueError("Tracks are not equal")
print(f"All first {i+1} tracks are equal") # type: ignore

Get scores

In [ ]:
my_scores: npt.NDArray = find_file(f"new_scores_{event_name}", dir=DIRECTORY, fallback_func=_make_scores, force_fallback=not preload)  # type: ignore
outrunner_scores: npt.NDArray = find_file(f"outrunner_scores_{event_name}", dir=DIRECTORY)  # type: ignore

Compare a few scores by eye

In [ ]:
limit=10
print(my_scores[:limit])
print(outrunner_scores[:limit])

Verify all scores

In [ ]:
print("Equal: ", verify_matrices(my_scores, outrunner_scores))

Get merged

In [ ]:
merged_tracks: npt.NDArray = find_file(f"merged_tracks_{event_name}", dir=DIRECTORY, fallback_func=_make_merged_tracks, force_fallback=not preload)  # type: ignore

# Make submission

Make submission `DataFrame`

In [ ]:
submission = find_file(
    f"submission_{event_name}", dir=DIRECTORY, fallback_func=_make_submission, force_fallback=not preload
)  # type: ignore

Evaluate submission

In [ ]:
score = my_code.score_event(truth, submission)
print("TrackML Score:", score)
print("Fast score: ", my_code.score_event_fast(submission, truth))

Add our track_id to truth

In [ ]:
combined: pd.DataFrame = truth[["hit_id", "particle_id", "weight", "tx", "ty", "tz"]].merge(submission, how="left", on="hit_id") # type: ignore
# Group by unique combinations of track_id (our) and particle_id (truth); count number of hits overlapping
grouped: pd.DataFrame = (
    combined.groupby(["track_id", "particle_id"]).hit_id.count().to_frame("count_both").reset_index()
)
# Skip unallocated tracks (track_id == 0)
print(grouped[grouped['track_id'] > 0])

# Show some reconstructions

Select most likely related true `particle_id`

In [ ]:
# Tracks are already ordered by score
track_id = 2000
possible_particle_ids: pd.DataFrame = grouped[grouped["track_id"] == track_id].sort_values(
    "count_both", ascending=False
)
most_likely_particle_id = int(possible_particle_ids.iloc[0]["particle_id"])

Select related truth and reconstructed data

In [ ]:
reconstructed_track = combined[combined["track_id"] == track_id]
truth_track = combined[combined["particle_id"] == most_likely_particle_id]

In [ ]:
print("Selected track ids: \n", reconstructed_track['hit_id'].values)

In [ ]:
print(reconstructed_track)

In [ ]:
reconstructed_weight_total = reconstructed_track["weight"].sum()
reconstructed_weight_overlap = reconstructed_track[reconstructed_track['particle_id'] == most_likely_particle_id]['weight'].sum()

truth_weight = truth_track["weight"].sum()

ratio = reconstructed_weight_overlap / truth_weight

print (f"Track {track_id} has total weight {reconstructed_weight_total}, vs {truth_weight} from particle {most_likely_particle_id}, ratio: {ratio:.4f}")

Show fig

In [ ]:
fig: plt.Figure = my_code.plot_prediction(truth_track, reconstructed_track, most_likely_particle_id, label_type="particle_id")
fig.suptitle(f"Track {track_id} with particle id {most_likely_particle_id} \n\
             weight ratio: {ratio:.2f}\
             ", fontsize=20)

Save fig

In [ ]:
do_save_fig = False
if do_save_fig:
    fig.savefig(f"reconstructed_track_{track_id}_{event_name}.png", dpi=300)
    plt.close()

# Link reconstructed tracks with true tracks and add primary vertex information

Define minimal hits for truth and reconstructed tracks to be considered

In [ ]:
min_hits = 4

Count reconstructed tracks hits

In [ ]:
r_count = combined[combined['track_id']!=0].value_counts('track_id').to_frame('count_reco').sort_index()
r_considered = r_count[r_count['count_reco'] >= min_hits]
r_considered


Filter discarded tracks

In [ ]:
valid_tracks = grouped[(grouped['particle_id'] != 0) & (grouped['track_id'] != 0) & (grouped['count_both'] > 1)].reset_index(drop=True)
valid_tracks

Count trutht tracks hits

In [ ]:
p_count = truth[truth['particle_id'] != 0].groupby('particle_id').count().hit_id.sort_values(ascending=False).to_frame('count_truth')
p_considered = p_count[p_count['count_truth'] >= min_hits]
p_considered

Confirm too many matches for amount of particles, should consider only primary particles

In [ ]:
print("Number of matches: ", len(valid_tracks))
print("Number of particles",  len(truth.particle_id.unique()))

Combine counts on considered truth and reco tracks

In [ ]:
considered_pairs = valid_tracks[valid_tracks['particle_id'].isin(p_considered.index)][valid_tracks['track_id'].isin(r_considered.index)].sort_values(['count_both', 'track_id'], ascending=[False, True]).reset_index(drop=True).merge(p_considered, on='particle_id').merge(r_considered, on='track_id')
considered_pairs

Find particles linked to too many reco tracks

In [ ]:
duplicates_mask = considered_pairs[considered_pairs.duplicated(subset='particle_id', keep='first')].sort_index()
duplicates_mask


Select majority track for reco's

In [ ]:
primary_matched =  considered_pairs[~considered_pairs.index.isin(duplicates_mask.index)]
primary_matched

Define 'good' tracks

In [ ]:
track_purity = primary_matched['count_both'] / primary_matched['count_reco']
track_purity

In [ ]:
particle_purity = primary_matched['count_both'] / primary_matched['count_truth']
particle_purity

In [ ]:
primary_matched.insert(4, 'particle_purity', particle_purity)
primary_matched.insert(6, 'track_purity', track_purity)

In [ ]:
primary_matched

### _Both ratios have to be above 50% to define a good track so that a one-to-one relationship between particle and track can be defined._

In [ ]:
good_tracks = primary_matched[primary_matched['particle_purity'] >= 0.5]
good_tracks

In [ ]:
efficiency = len(good_tracks) / len(p_considered)
print(f"Efficiency: {100*efficiency:.2f}%")

#### Find primary vertex

In [ ]:
r_all = np.sqrt(np.sum(truth[['tx', 'ty', 'tz']].values**2, axis=1))
truth.insert(2, 'r', r_all)
truth

In [ ]:
r_sorted = truth.sort_values('r', ascending=True)
r_sorted

In [ ]:
r_mask = r_sorted[r_sorted.duplicated(subset='particle_id', keep='first')]
r_mask

In [ ]:
r_0 = r_sorted[~r_sorted.index.isin(r_mask.index)][['particle_id','r']].rename(columns={'r': 'r_0'})
r_0

Define primary vertex hit values

In [ ]:
primary_vertex_index = r_0.index
primary_vertex_data = truth.loc[primary_vertex_index]
primary_vertex_data.rename(columns={'r':'r_0','tx': 'x_0', 'ty': 'y_0', 'tz': 'z_0', 'tpx': 'px_0', 'tpy':'py_0', 'tpz': 'pz_0', 'weight':'weight_0'}, inplace=True)
primary_vertex_data

In [ ]:
primary_vertex_data['p_0']=np.sqrt(primary_vertex_data['px_0']**2+primary_vertex_data['py_0']**2+primary_vertex_data['pz_0']**2)
primary_vertex_data['p_t_0']=np.sqrt(primary_vertex_data['px_0']**2+primary_vertex_data['py_0']**2)
primary_vertex_data['log_10_p_t_0']=np.log10(primary_vertex_data['p_t_0'])
primary_vertex_data['phi_0'] = np.arctan2(primary_vertex_data['y_0'], primary_vertex_data['x_0'])
primary_vertex_data['theta_0'] = np.arccos(primary_vertex_data['z_0']/primary_vertex_data['r_0'])
primary_vertex_data['pseudo_rapidity_0'] = -np.log(np.tan(primary_vertex_data['theta_0']/2))
primary_vertex_data

In [ ]:
good_tracks = good_tracks.merge(primary_vertex_data, on='particle_id')
good_tracks

In [ ]:
p_considered = p_considered.merge(primary_vertex_data, on='particle_id')
p_considered

## Plot histograms of primary vertex information

In [ ]:
def compare_histograms(truth, test, variable: str | None = None,bins=100, range=None, density=False, title=None, xlabel=None, ylabel=None, figsize=(10, 6), **kwargs):
    fig, ax = plt.subplots(figsize=figsize)
    ax.hist([truth[variable], test[variable]], bins=bins, range=range, density=density, alpha=0.75,**kwargs)
    ax.set_title(title)
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    plt.legend()
    return fig

In [ ]:
variables = ['r_0', 'p_0', 'p_t_0', 'log_10_p_t_0', 'phi_0', 'theta_0', 'pseudo_rapidity_0']
var_labels = ['vertex $r_0$ [mm]', '$p$', '$P_{T}$', '$log_{10}$ $p_{T}$', '$\\phi$', '$\\theta$', '$\\eta$']

In [ ]:
for variable, label in zip( variables, var_labels):
    fig = compare_histograms(p_considered, good_tracks, variable=variable, bins=100,title=f'Primary vertex {label}', xlabel=label, ylabel='Frequence', label=['Truth', 'Reconstructed'], histtype='step', linewidth=1,color=['blue', 'orange'])

# Calculate efficiency over primary vertex variables

In [ ]:
def efficiency(truth: pd.DataFrame, test: pd.DataFrame, variable: str | None = None, bins: int=100, min=None, max=None, title=None, xlabel='x', ylabel='Efficiency', figsize=(10, 6), **kwargs):
    min_bin = truth[variable].min() if min is None else min
    max_bin = truth[variable].max() if max is None else max
    bins_index = pd.cut(pd.Series([min_bin, max_bin]), bins=bins, retbins=True)[1]
    print(label, min_bin, max_bin)
    
    cut_truth = pd.cut(truth[variable], bins=bins_index)  # type: ignore
    groups_truth = truth.groupby(cut_truth)
    binned_truth = groups_truth.count()[variable]
    
    cut_test = pd.cut(test[variable], bins=bins_index)  # type: ignore
    groups_test = test.groupby(cut_test)
    binned_test = groups_test.count()[variable]
    
    efficiency =  binned_test/binned_truth
    
    fig, ax = plt.subplots(figsize=figsize)
    x = np.arange(min_bin, max_bin, (max_bin-min_bin)/bins)
    yerr = np.sqrt(binned_test)/binned_truth
    ax.plot(x, efficiency.values, label=ylabel, **kwargs)
    ax.fill_between(x, efficiency-yerr, efficiency + yerr, alpha=0.5, label='Error')
    ax.set_ylim(0, 1)
    ax.set_ylabel(ylabel)
    ax.set_xlabel(xlabel)
    ax.set_title(f'Efficiency by {xlabel}' if title is None else title)
    ax.legend()
    
    return fig

In [ ]:
variables = ['r_0', 'r_0', 'p_0', 'p_t_0', 'log_10_p_t_0', 'phi_0', 'theta_0', 'pseudo_rapidity_0']
var_labels = ['vertex $r_0$ [mm]','zoom vertex $r_0$ [mm]', '$p$', '$P_{T}$', '$log_{10}$ $p_{T}$', '$\\phi$', '$\\theta$', '$\\eta$']
mins = [0, 0,None, 0, None, -np.pi, -np.pi, -np.pi]
maxs = [600,100,25, 5, 2,np.pi, np.pi, np.pi]
for variable, label, min, max in zip( variables, var_labels, mins, maxs):
    fig = efficiency(p_considered, good_tracks, variable=variable, bins=100,min=min, max=max, xlabel=label)